In [1]:
#import libraries
import requests
import pandas as pd
import json

In [2]:
#get api data from the US Patent API
url_one = 'https://api.patentsview.org/patents/query?q={"_and":[{"_gte":{"patent_date":"2020-01-01"}},{"assignee_state":"Fl"}]}&f=["patent_number","patent_year","patent_date", "patent_title", "patent_kind", "patent_num_combined_citations", "inventor_state_fips","inventor_county_fips", "assignee_type", "assignee_county", "assignee_city","assignee_lastknown_city", "assignee_lastknown_country", "assignee_organization", "cpc_section_id", "cpc_subsection_id"]&s=[{"patent_date":"desc"}]&o={"per_page":10000,"page":1}'
response_one = requests.get(url_one)
fl_patents_one = response_one.json()

url_two = 'https://api.patentsview.org/patents/query?q={"_and":[{"_gte":{"patent_date":"2020-01-01"}},{"assignee_state":"Fl"}]}&f=["patent_number","patent_year","patent_date", "patent_title", "patent_kind", "patent_num_combined_citations", "inventor_state_fips","inventor_county_fips", "assignee_type", "assignee_county", "assignee_city", "assignee_lastknown_city", "assignee_lastknown_country", "assignee_organization", "cpc_section_id", "cpc_subsection_id"]&s=[{"patent_date":"desc"}]&o={"per_page":10000,"page":2}'
response_two = requests.get(url_two)
fl_patents_two = response_two.json()

In [3]:
patent_data_one = []
for patent in fl_patents_one["patents"]:
    base_data = {
        "patent_number": patent["patent_number"],
        "patent_title": patent["patent_title"],
        "patent_year": patent["patent_year"],
        "patent_date": patent["patent_date"],
        "patent_kind": patent["patent_kind"],
        "patent_num_combined_citations": patent["patent_num_combined_citations"]
    }
    
    # expand inventors
    for inventor in patent["inventors"]:
        inventor_data = base_data.copy()
        inventor_data.update({
            "inventor_state_fips": inventor["inventor_state_fips"],
            "inventor_county_fips": inventor["inventor_state_fips"]
        })
        patent_data_one.append(inventor_data)
        
    # Expand assignees
    for assignee in patent["assignees"]:
        assignee_data = base_data.copy()
        assignee_data.update({
            "assignee_type": assignee["assignee_type"],
            "assignee_county": assignee["assignee_county"],
            "assignee_city": assignee["assignee_city"],
            "assignee_lastknown_city": assignee["assignee_lastknown_city"],
            "assignee_lastknown_country": assignee["assignee_lastknown_country"],
            "assignee_organization": assignee["assignee_organization"],
            "assignee_key_id": assignee["assignee_key_id"]
        })
        patent_data_one.append(assignee_data)
        
    # Expand CPCs
    for cpc in patent["cpcs"]:
        cpc_data = base_data.copy()
        cpc_data["cpc_section_id"] = cpc["cpc_section_id"],
        cpc_data["cpc_subsection_id"] = cpc["cpc_subsection_id"]
        patent_data_one.append(cpc_data)


# Create DataFrame
fl_patents_cleaned_one = pd.DataFrame(patent_data_one)

In [4]:
patent_data_two = []
for patent in fl_patents_two["patents"]:
    base_data = {
        "patent_number": patent["patent_number"],
        "patent_title": patent["patent_title"],
        "patent_year": patent["patent_year"],
        "patent_date": patent["patent_date"],
        "patent_kind": patent["patent_kind"],
        "patent_num_combined_citations": patent["patent_num_combined_citations"]
    }
    
    # expand inventors
    for inventor in patent["inventors"]:
        inventor_data = base_data.copy()
        inventor_data.update({
            "inventor_state_fips": inventor["inventor_state_fips"],
            "inventor_county_fips": inventor["inventor_state_fips"]
        })
        patent_data_two.append(inventor_data)
        
    # Expand assignees
    for assignee in patent["assignees"]:
        assignee_data = base_data.copy()
        assignee_data.update({
            "assignee_type": assignee["assignee_type"],
            "assignee_county": assignee["assignee_county"],
            "assignee_city": assignee["assignee_city"],
            "assignee_lastknown_city": assignee["assignee_lastknown_city"],
            "assignee_lastknown_country": assignee["assignee_lastknown_country"],
            "assignee_organization": assignee["assignee_organization"],
            "assignee_key_id": assignee["assignee_key_id"]
        })
        patent_data_two.append(assignee_data)
        
    # Expand CPCs
    for cpc in patent["cpcs"]:
        cpc_data = base_data.copy()
        cpc_data["cpc_section_id"] = cpc["cpc_section_id"],
        cpc_data["cpc_subsection_id"] = cpc["cpc_subsection_id"]
        patent_data_two.append(cpc_data)


# Create DataFrame
fl_patents_cleaned_two = pd.DataFrame(patent_data_two)

In [5]:
print(fl_patents_cleaned_two.tail())

      patent_number       patent_title  patent_year patent_date patent_kind  \
49569       D872306        Door facing         2020  2020-01-07          S1   
49570       D872375  Fire extinguisher         2020  2020-01-07          S1   
49571       D872375  Fire extinguisher         2020  2020-01-07          S1   
49572       D872375  Fire extinguisher         2020  2020-01-07          S1   
49573       D872375  Fire extinguisher         2020  2020-01-07          S1   

       patent_num_combined_citations inventor_state_fips inventor_county_fips  \
49569                             59                 NaN                  NaN   
49570                              3                  37                   37   
49571                              3                  37                   37   
49572                              3                 NaN                  NaN   
49573                              3                 NaN                  NaN   

      assignee_type assignee_county   

In [6]:
#remove parenthesis and commas from cpc_section_id and from cpc_subsection_id
fl_patents_cleaned_one["cpc_section_id"] = fl_patents_cleaned_one["cpc_section_id"].astype(str).str.replace(r"[(),'']", "", regex=True)
fl_patents_cleaned_one["cpc_subsection_id"] = fl_patents_cleaned_one["cpc_subsection_id"].astype(str).str.replace(r"[(),'']", "", regex=True)
fl_patents_cleaned_two["cpc_section_id"] = fl_patents_cleaned_two["cpc_section_id"].astype(str).str.replace(r"[(),'']", "", regex=True)
fl_patents_cleaned_two["cpc_subsection_id"] = fl_patents_cleaned_two["cpc_subsection_id"].astype(str).str.replace(r"[(),'']", "", regex=True)

In [7]:
print(fl_patents_cleaned_one.tail())

      patent_number                                       patent_title  \
56985      11294930  Resource scaling for distributed database serv...   
56986      11294930  Resource scaling for distributed database serv...   
56987      11294930  Resource scaling for distributed database serv...   
56988      11294930  Resource scaling for distributed database serv...   
56989      11294930  Resource scaling for distributed database serv...   

       patent_year patent_date patent_kind  patent_num_combined_citations  \
56985         2022  2022-04-05          B2                              7   
56986         2022  2022-04-05          B2                              7   
56987         2022  2022-04-05          B2                              7   
56988         2022  2022-04-05          B2                              7   
56989         2022  2022-04-05          B2                              7   

      inventor_state_fips inventor_county_fips assignee_type assignee_county  \
56985       

In [8]:
#merge dataframes
fl_patents_cleaned = pd.concat([fl_patents_cleaned_one, fl_patents_cleaned_two], axis=0)

In [27]:
print(fl_patents_cleaned.tail())

      patent_number       patent_title  patent_year patent_date patent_kind  \
49569       D872306        Door facing         2020  2020-01-07          S1   
49570       D872375  Fire extinguisher         2020  2020-01-07          S1   
49571       D872375  Fire extinguisher         2020  2020-01-07          S1   
49572       D872375  Fire extinguisher         2020  2020-01-07          S1   
49573       D872375  Fire extinguisher         2020  2020-01-07          S1   

       patent_num_combined_citations inventor_state_fips inventor_county_fips  \
49569                             59                 NaN                  NaN   
49570                              3                  37                   37   
49571                              3                  37                   37   
49572                              3                 NaN                  NaN   
49573                              3                 NaN                  NaN   

      assignee_type assignee_county   

In [28]:
fl_patents_cleaned.to_csv('data\fl_patents_cleaned.csv')

OSError: [Errno 22] Invalid argument: 'data\x0cl_patents_cleaned.csv'